Modified: Jul 17, 2019
# Curve Representation using Levelset function
- Signed Distance Function


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
from scipy.signal import correlate2d
import pandas as pd
    
from pathlib import Path
from pprint import pprint as pp
p = print

from functools import lru_cache
from collections import defaultdict

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts, dim
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams


import panel as pn
from bokeh.palettes import GnBu9


hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

# pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint, timeit
import utils as u

import sdfs 
from vector import Vector as vec
from samples import LSTestSample

### Set visualization options

In [ ]:
H, W = 500,500

In [ ]:
opts.defaults(
    opts.Image(colorbar=True, active_tools=['wheel_zoom'], tools=['hover']),
    opts.Curve(tools=['hover'], active_tools=['wheel_zoom']),
    opts.RGB(active_tools=['wheel_zoom'], tools=['hover'])
)

In [ ]:
img_opts = opts.Image(height=H, width=W)
vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
#                                magnitude=dim('Magnitude').norm()*0.2,
                               pivot='tip',
                               rescale_lengths=True)
curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
                         legend_position='bottom',
#                             xlim=(-10,10), ylim=(-10,10),
#                         color=dim('p')*256-50
                        )
contour_opts = opts.Contours(width=W, height=H, 
                             colorbar=False,
                             legend_position='bottom',
                             tools=['hover'])

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

In [ ]:
import streamz
import streamz.dataframe

To deinf a curve on a plane (ie. planary curve) we need
- parameter, eg. p $\in [0,1]$
- two functions $x(p)$ and $y(p)$, which define the coordinate of the point $C(p)$ in x and y axis, respectively

## Signed Distance Function 
### For a circle

- $\phi(x,y)$ is the distance between point $p = (x,y)$ and the curve. Here the curve we want to represent is a unit circle
- $\phi(x,y)$ is Dist(origin, $p$) - Dist(origin, a point on the unit circle touched by a stright ray passing through the origin and $p$). The second term is always 1 for a unit circle. So we get the $\phi$ as following:

$ \phi(x,y) = \sqrt{(x^2+y^2)} - 1 $

Now let's visualize it using holoviews.

In [ ]:
def phi(x,y):
    """Signed Distance function for a unit circle"""
    return np.sqrt(x**2 + y**2) - 1


In [ ]:
n = 100
xs = np.linspace(-1.5,1.5,num=n)
ys = np.linspace(-1.5,1.5,num=n)[::-1] #flipped because I'm interested in the cartesian coordinate system's view
# nprint(xs,ys)

In [ ]:
X,Y = np.meshgrid(xs,ys)
Z = phi(X,Y)

In [ ]:
# See hv.Image?
cmap = ['twilight', 'RdBu','Spectral']
img = hv.Image((xs,ys,Z)).opts(img_opts).opts(cmap=cmap[1])
contour = hv.operation.contours(img, levels=1).opts(contour_opts).opts(cmap='gray') #love this level=0:D
img * contour
#or, equivalently hv.Image(Z, bounds=(xs.min(), ys.min(), xs.max(), ys.max())).opts(img_opts)#.opts(cmap='twilight')

In [ ]:
kernel = np.array([[-0.5,0,0.5]])
corr_opts = dict(mode='same')
Zx = correlate2d(Z,kernel, **corr_opts)
Zy= correlate2d(Z, kernel.T, **corr_opts)

(
    hv.Image((xs,ys,Zx)) + hv.Image((xs,ys,Zy))
).opts(img_opts);

In [ ]:
def img_contour(xs, ys, Z, cmap=None, show_contour=True, levels=1):
    if cmap is None:
        cmap=dict(img_cmap='RdBu', contour_cmap='gray')
    img = hv.Image((xs,ys,Z)).opts(img_opts).opts(cmap=cmap['img_cmap'])
    contour = hv.operation.contours(img, levels=levels).opts(contour_opts).opts(cmap=cmap['contour_cmap']) #love this level=0:D
    if show_contour:
        return img*contour
    return img

def UV2angMag(U,V):
    """
    U,V (MxN np.ndarray): encodes X,Y coordinate grids respectively
    Returns:
    - angle, mag: tuple of MxN np.ndarray that encode ang (or mag) for the grid space
    That means, angle[j][i] at (X[j][i],Y[j][i]) location
    """
    mag = np.sqrt(U**2 + V**2)
    angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)
#     angle =  np.arctan2(V,U)


    return (angle, mag)
    

In [ ]:
# img_contour(xs,ys,Zx)+ img_contour(xs,ys,Zy)#,show_contour=False, levels=5)
grad_angle, grad_mag = UV2angMag(Zx,Zy)
gradfield = hv.VectorField( (X,Y,grad_angle, grad_mag) ).opts(vfield_opts)#.opts(height=1000, width=1000)

# img.opts(height=1000,width=1000) * contour * gradfield


---
Modified: Jul 21, 2019

## Curve Evolution 
- [ ] Finite Difference method on parametric equations
- [ ] Levelset functions
    - 2D signed distance functions: [src](https://is.gd/t7p5mk)
- [ ] Active contour on satellite images
- [ ] Agent-based modelling with specified rules
    - satellite image segmentation (~ clustering based on local features)

In [ ]:
def eval_sdf(xs, ys, sdFunc):
    zz = np.empty( (len(ys), len(xs)) )
    
    for j in range(len(ys)):
        for i in range(len(xs)):
            q = vec(xs[i],ys[j])
            zz[j,i] = sdFunc(q)
    return zz

In [ ]:
CACHE = {}
HITS = defaultdict(int)

t = 0
xrange = (-1,1)
yrange=(-1,1)
n_points = 100
xs = np.linspace(*xrange,num=n_points)
ys = np.linspace(*yrange,num=n_points)

sdf = sdfs.sdUnitCircle
key = str((xrange, yrange, n_points, sdf))

try:
    zz = CACHE[key]
    HITS[key] += 1

except KeyError:
    zz = eval_sdf(xs, ys, sdf)
    CACHE[key] = zz

zz0_img = hv.Image( (xs, ys, zz), group='zz', label='t0' ).opts(img_opts)\
            .opts(xlim=xrange, ylim=yrange)
zz0_contour = hv.operation.contours(zz_img, levels=0, group='contour').opts(contour_opts) \
            .opts(xlim=xrange, ylim=yrange, cmap='gray')

# compute gradients
gradx = correlate2d(zz, kernel, mode='same')
grady = correlate2d(zz, kernel.T, mode='same')

ang, mag = u.UV2angMag(gradx, grady)

gradfield = hv.VectorField((xs, ys, ang, mag)).opts(vfield_opts)
gradmag_img = hv.Image( (xs,ys,np.abs(mag)), group='grad', label='t0' ).opts(img_opts).opts(xlim=xrange, ylim=yrange)

overlay = datashade(zz0_img, cmap=GnBu9) * zz0_contour * gradfield + gradmag_img

# overlay
img0 = zz0_img *zz0_contour 
grad0 = gradmag_img * zz0_contour
img0+grad0

In [ ]:
zz1 = zz - np.abs(mag)

In [ ]:
zz1_img = hv.Image( (xs,ys,zz1), group='zz', label='t1' ).opts(img_opts).opts(xlim=xrange, ylim=yrange)
zz1_contour = hv.operation.contours(zz1_img, levels=0, group='contour').opts(contour_opts) \
            .opts(xlim=xrange, ylim=yrange, cmap='gray')

In [ ]:
img1 = zz1_img *zz1_contour
(img0 + img1)

In [ ]:
hv.NdOverlay({1:zz0_contour, 2:zz1_contour})

---
### Step function to evolve the levelset equation for curve 
General curve evolution can be simulated as time integration process of the solving the following `initial value problem` for a general PDE:

$$
\frac{\partial \phi}{\partial t} = L(t, \phi(t)), ~~~~ \phi(t_0) = \phi^{0}
$$

Note that we will use $\frac{\partial \phi}{\partial t}$ and $\phi_t$ interchangably, to denote the time derivative of $\phi$.


The first-order Taylor expansion about $(t+\Delta t)$ gives,

$$
\phi( t+\Delta t) \approx \phi(t) + \Delta t \phi_{t}(t)
$$

By the [CFL](#) condition, if $L$ depends on at most the first-order derivatives (eg. $\nabla \phi$, the spatial derivative of $\phi$), then 

First let's implement the curve evolution according to this PDE:

$$
\phi_{t}(x,y,t) = \lVert \nabla \phi(x,y,t) \rVert
$$

1. Inputs
    - phi (MxN) at current time
2. Computation
    - compute gradx, grady -> mag_grad
    - next_phi = phi - abs(mag_grad)
        - make hv.image of next_phi
        - make hv.contour of next_phi
        - save the contour into a list
    - after all iterations, make ndOverlay from the list of contours -> thish will show the unit circle's evolution according to:

$$
\phi_{t}(x,y,t) = \left| \nabla \phi(x,y,t) \right|
$$

In [ ]:
def step(xs, ys, zz, return_grad=True, **kwargs):
    """
    One step of curve evolution of levelset function. 
    In other words, this function defins a discrete version of PDE for a curve evolution 
    Returns current zz's gradients and next zz
    
    Args:
    - xs (1D array of length N): x-coordinates
    - ys (1D array of length M): y-coordinates
    - zz (MxN np.ndarray): values of the levelset function s.t.
        zz[j][i] = phi(x=xs[i], y=ys[j])
        
    - kwargs (dict) may have the following (key, value) pairs:
        - "step_idx" -> (int) : indicating the iteration index following the (discreate) curve evolution equation
    
    Returns:
    - a tuple of MxN np.ndarray(s): 
        If return_grad is True, returns (grad_ang, grad_mag, next_zz), or (next_zz)
        where next_zz encodes the levelset values after a single step of curve evolution
    """
    
    # compute gradients
    gradx = correlate2d(zz, kernel, mode='same')
    grady = correlate2d(zz, kernel.T, mode='same')

    grad_ang, grad_mag = u.UV2angMag(gradx, grady)
    next_zz = zz - np.abs(grad_mag)
    if return_grad:
        return (grad_ang, grad_mag, next_zz)
    else:
        return (next_zz)

### Take 1

In [ ]:
# Initialize curve evolution
## First, define xs, ys ,and initial zz
CACHE = {}
HITS = defaultdict(int)

xrange = (-1,1)
yrange=(-1,1)
n_points = 100
xs = np.linspace(*xrange,num=n_points)
ys = np.linspace(*yrange,num=n_points)

# sdf = sdfs.sdUnitCircle
sdf = sdfs.sdUnitHline

zz = eval_sdf(xs, ys, sdf)

n_steps = 10
step_idx = 0

contours = {}

cmaps_filtered = ['Reds','Oranges', 'YlOrRd', 'Greens', 'Blues', 'PuBuGn', 'PuRd', 'Purples', 'Greys']
while step_idx < n_steps:
    zz_img = hv.Image( (xs, ys, zz), group=f'{step_idx}' ).opts(img_opts) \
                .opts(xlim=xrange, ylim=yrange)
    zz_contour = hv.operation.contours(zz_img, levels=0).relabel(group=f'{step_idx}').opts(contour_opts) \
                .opts(xlim=xrange, ylim=yrange, cmap=cmaps_filtered[step_idx%len(cmaps_filtered)])

    contours[step_idx] = zz_contour

    # get current gradients and update levelset to next step
    grad_ang, grad_mag, next_zz = step(xs, ys, zz)
    gradfield = hv.VectorField((xs, ys, grad_ang, grad_mag), group=f'{step_idx}').opts(vfield_opts)

    zz = next_zz
    step_idx += 1
nd_contours = hv.NdOverlay(contours, group='Contours', kdims='step')#.opts(hv.opts('Contours', color='step'))
nd_contours;

### Take 2: Better coloring

In [ ]:
# Initialize curve evolution
## First, define xs, ys ,and initial zz
CACHE = {}
HITS = defaultdict(int)

xrange = (-5,5)
yrange=(-5,5)
n_points = 100
xs = np.linspace(*xrange,num=n_points)
ys = np.linspace(*yrange,num=n_points)

# sdf = sdfs.sdUnitCircle
sdf = sdfs.sdUnitHline
zz = eval_sdf(xs, ys, sdf)

n_steps = 20
step_idx = 0

contours = {}
while step_idx < n_steps:
    zz_img = hv.Image( (xs, ys, zz), group=f'{step_idx}' ).opts(img_opts) \
                .opts(xlim=xrange, ylim=yrange)
    zz_contour = hv.operation.contours(zz_img, levels=0).relabel(group=f'{step_idx}').opts(contour_opts) \
                .opts(xlim=xrange, ylim=yrange)

    contours[step_idx] = zz_contour

    # get current gradients and update levelset to next step
    grad_ang, grad_mag, next_zz = step(xs, ys, zz)
    gradfield = hv.VectorField((xs, ys, grad_ang, grad_mag), group=f'{step_idx}').opts(vfield_opts)

    zz = next_zz
    step_idx += 1


# Put each (single-layer) contour line together into a single contours element
all_contours = hv.Contours([contour.add_dimension('step', 1, i, vdim=True) 
                            for i, contour in contours.items()], vdims=['z', 'step']).opts(contour_opts) 
all_contours.opts(cmap='Greens_r',color='step')

In [ ]:
# https://github.com/Ramesh-X/ModelViewer

---
Please refer to this [thesis](#) for more details

## Fundamental level-set equations
 
$$
\begin{align}
\frac{\partial \phi}{\partial{t}} &= -\nabla{\phi} \cdot \vec{V}  \label{eq:4.7}  \tag{4.7} \\
                                  &= \lVert \vec{\nabla} \phi \rVert F(\vec{x}, \vec{n}, \phi, ... )
                                     \label{eq:4.8}  \tag{4.8} \\
\end{align}
$$

Key:
- $Eqn. \ref{eq:4.7}$: describes the transportation of the interface (ie. curve) in an external vector field
- $Eqn. \ref{eq:4.8}$: describes the motion of the interface in its normal direction by a magnitude determined by the speed function, $F$
    - $F = $ const 
    - $F = - \alpha \kappa$ for some $ \alpha > 0$ 


## <mark> Stability of the solution </mark> [todo]
### 1. Choice of discrete spatial derivative -- forward, backward, central

- If the levelset equation depends on at most order 1 derivatives:

    For example, 
    $$
    \begin{align}
    F &= 1 \\
    \Rightarrow \frac{\partial \phi}{\partial{t}} &= \lVert \vec{\nabla} \phi \rVert
    \end{align}
    $$
    
    

    
- If the levelset equation depends on derivatives of order $\geq 2$:

    For example, 
    $$
    \begin{align}
    F &= - \frac{1}{\lVert \vec{\nabla} \phi \rVert} \kappa \\
    \Rightarrow \frac{\partial \phi}{\partial{t}} &=  \kappa
    \end{align}
    $$
    Recall that the curvature $\kappa$ involves second derivatives of $\phi$

## Perspective 1: solve \ref{eq:4.7} as an advection problem
- **Advection**: transportation of the interface in an external vector field

One way to implement the dynamic curve evolution is to view the curve (or surface in 3D) as a floating object in some flow field $\vec{V}$ and solve the time integration of an **initial-valued problem** of the following PDE.  In continuous time domain, this is expressed as:

$$
\frac{\partial \phi}{\partial t} = - \vec{\nabla} \phi \cdot \vec{V}  
$$

where the vector $\vec{\nabla} \phi$ refers to the spatial gradient of $\phi$. Note that a vector normal to any levelset satisfies:

$$\vec{n} = \frac{\vec{\nabla} \phi}{ \lVert \vec{\nabla} \phi \rVert }$$

Therefore, $\ref{eq:4.7}$ is projecting each flow vector at $(x,y)$ on the grid to $\vec{n}(x,y)$, which it takes into account just the component of the flow in the direction of the normal. 

### Up-winding 
- as a way to choose the correct direction of spatial derivative 

Let's denote the spatial gradient of $\phi$ as $\phi_{\vec{x}} = (\phi_x, \phi_y)$.
- The information need to compute the accurate spatial derivatives, $\phi_x, \phi_y$, comes from the **reverse direction** of the vector field flow 

## Implement discrete gradients 
Assume the input array is in Cartesian coordinate (rather than image coordinate or numpy array indexing order)

- mode: forward, backward, central
- axis: 0 (y-axis), 1 (x-axis)

Use scipy `correlated2d`

### First, gradient in x direction

In [ ]:
backwardx_kernel = np.atleast_2d([-1,1,0] )
forwardx_kernel = np.atleast_2d([0,-1,1] )
# unnecessary operation will be incurred because of the last zero, 
#but this ensures the center of kernel is overlaid on the location of interest
test = np.array( [[1,2,5,10,100], 
                  [0,-1,10,-3,9],
                  [100,-20, 8, 10,-10]]
               )
gradx_backward = correlate2d(test, backwardx_kernel, mode='same')
gradx_forward = correlate2d(test, forwardx_kernel, mode='same')

nprint('original', test)
nprint('gradx back: ', gradx_backward)
nprint('gradx forward: ', gradx_forward)

Look at the top-left element in the backward gradient in x (`gradx back`). Since it corresponds to the location in xy-plane at x=0, there is no previous value to compute its backward difference from. We can handle this case in several different ways:

- use the value as is: this is equialent to pad zero as the leftmost column. use boundary='fill' and fillvalue=0
- use the forward difference: this is equivalent to use boundary='symm' 

The first method is what we tried above since it's the default setting of `correlate2d`. Let's check the second method now.

In [ ]:
gradx_backward = correlate2d(test, backwardx_kernel, mode='same', boundary='symm')
gradx_forward = correlate2d(test, forwardx_kernel, mode='same', boundary='symm')

nprint('original', test)
nprint('gradx back: ', gradx_backward)
nprint('gradx forward: ', gradx_forward)

Hmm, so it seems like 'symm' option actually copies starting as the edge column. This is not the forward difference. With our `backwardx_kernel`, this will always result zero column. So let's instead just replace the first column of the `gradx_backward` with the first column of the `gradx_forward`.  Similarly, we can handle the other edge case (ie. the last column of the output `gradx_forward` where the forward value to take the forward difference with do not exist, by placing it with the backward difference values.


In [ ]:
backwardx_kernel = np.atleast_2d([-1,1,0] )
forwardx_kernel = np.atleast_2d([0,-1,1] )

test = np.array( [[1,2,5,10,100], 
                  [0,-1,10,-3,9],
                  [100,-20, 8, 10,-10]]
               )
dxb = correlate2d(test, backwardx_kernel, mode='same')
dxf = correlate2d(test, forwardx_kernel, mode='same')

nprint('original', test)
nprint('gradx back: ', dxb)
nprint('gradx forward: ', dxf)

In [ ]:
# Now replace the first column of backwardx with the first column of forwardx 
# And replace the last column of the forwardx with the last column of backwardx
dxb[:,0] = dxf[:,0]
dxf[:,-1] = dxb[:,-1]

In [ ]:
dxb

In [ ]:
dxf

In summary,


In [ ]:
backwardx_kernel = np.atleast_2d([-1,1,0] )
forwardx_kernel = np.atleast_2d([-1,1] )

@timeit
def gradx(M, switch):
    dxb = correlate2d(M, backwardx_kernel, mode='same')
    dxf = correlate2d(M, forwardx_kernel, mode='same')

    if switch:
        dxb[:,0] = dxf[:,0]
        dxf[:,-1] = dxb[:,-1]

    return dxb, dxf

In [ ]:
def test_gradx():
    test = np.array( [[1,2,5,10,100], 
                  [0,-1,10,-3,9],
                  [100,-20, 8, 10,-10]]
                   )
    switch = True
    dxb, dxf = gradx(test, switch)
    nprint('original', test)
    nprint('gradx back: ', dxb)
    nprint('gradx forward: ', dxf)
test_gradx()

### Similarly, graident in y
Be careful because numpy array's row indexing order is the opposite of Cartesian y-axis's direction

In [ ]:
backwardy_kernel = np.atleast_2d([1,-1]).T
forwardy_kernel = np.atleast_2d([1,-1,0]).T
grady_forward = correlate2d(test, forwardy_kernel, mode='same')
grady_backward = correlate2d(test, backwardy_kernel, mode='same')
nprint('original: ', test)
nprint('forward y', grady_forward)
nprint('backward y', grady_backward)

- Switching in y-direction

When computing backward grady, elements in the last row does not have valid previous values to compute the backward difference. So we replace the last row of the `grady_backward` with the last row of the `grady_forward`

In [ ]:
grady_backward[-1] = grady_forward[-1]

Similarly, the first row of the forward_y does not have valid next values to compute the forward difference with. So we replace the first row of the `grady_forward` with the first row of the `grady_backward`

In [ ]:
grady_forward[0] = grady_backward[0]
nprint('grady backward: ', grady_backward)
nprint('grady forward: ', grady_forward)

In [ ]:
@timeit
def grady(M, switch):
    dyb = correlate2d(M, backwardy_kernel, mode='same')
    dyf = correlate2d(M, forwardy_kernel, mode='same')

    if switch:
        dyb[-1] = dyf[-1]
        dyf[0] = dyb[0]

    return dyb, dyf

In [ ]:
def test_grady():
    M = test
    dyb, dyf = grady(M,True)
    nprint('original: ', M)
    nprint('backword grady: ', dyb)
    nprint('forward grady: ', dyf)
test_grady()

---
### Summary on how to compute spatial gradients
Kernels for spatial gradients of $\phi$
- mode: forward, backward
- directions: x and y 
- Dxp (forward gradx),Dxm (backward gradx), Dyp, Dym 

---
Modified: Jul 26, 2019
### [SIDE] Computing Time Comparison 
Element-by-element finite difference vs. correlation operator

In [ ]:
@timeit
def diff_naive(M):
    """
    Returns
    a tuple of backward and forward difference np.ndarray
    """
    
    Dxf = np.empty_like(M)
    Dxb = np.empty_like(M)

    h,w = M.shape
    for j in range(h):
        for i in range(1, w-1):
            Dxf[j,i] = M[j,i+1] - M[j,i]
            Dxb[j,i] = M[j,i] - M[j,i-1]
        # First column doesn't have valid values take the backward difference with
        # Instead, take forward difference
        Dxf[j,0] = M[j,1] - M[j,0]
        Dxb[j,0] = M[j,1] - M[j,0]

        # Last column doesn't have valid values take the forward difference with
        # Instead, take backward difference
        Dxf[j, w-1]= M[j,w-1] - M[j,w-2]
        Dxb[j, w-1]= M[j,w-1] - M[j,w-2]

    return Dxb, Dxf
def test_diff_naive():
    M = test
    dxb, dxf = diff_naive(M)
    nprint(dxb, dxf)
test_diff_naive()

In [ ]:
M = np.random.randn(1000,1000)

In [ ]:
out1 = diff_naive(M);

In [ ]:
out2 = gradx(M,True);

In [ ]:
np.array_equal(out1, out2)

In [ ]:
# np.gradient

oh wow. These is a big speedup from vectorizing the computatation. Let's go with the correlated2d based function. 

---
# LevelSet Class


In [ ]:
import calculus as calc
from grid import CartesianGrid
from samples import LSTestSample

In [ ]:
class LevelSet():
    """LevelSet Evolution according to an initial-valued problem given by a PDE
    Args:
    - F (callable): takes a LevelSet object and time index and returns a np array 
    with the same shape as the levelset's grid
    """
    def __init__(self, grid=None, grid_step=1., t=0):

        if grid is None:
            grid = self.get_test_grid()
        self.grid = grid
        self.h, self.w = grid.shape
          
        self.grid_step = grid_step # grid step size
        self.delta = np.inf # average change in grid values
        self.t = t# current time        
            
    def initialize_grid(self):
        pass
    
    def run(self, F, dt, pde_class, threshold=1e-3, maxIter=1e4):
        count = 0
        deltas = []
        grids = {}
        while self.delta > threshold:
            if count > maxIter: 
                print("MaxIter reached: ", count)
                break
            self.propagate(F,dt,pde_class)
            deltas.append(self.delta)
            count += 1
            if count%1000:
                grids[self.t] = self.grid
        print(f"Ran for {count} steps, for {self.t} periods")
        print(f"\taverage delta phi: {self.delta}")
        return deltas, grids

    def propagate(self, F, dt, pde_class):
        """
        Equation 4.8 and 4.20
        For stability in computing the spatial gradients, use Eqn. 4.33
        
        pde_class (str): 'hyperbolic', 'parabolic'
            - 1: if F depends on at most order 1 derivatives of the levelset function phi 
            wrt space and time, the information propagation has a certain direction 
            (ie. "characteristics"), and we need to be careful about which gradient to 
            take -- backward? forward?  In this case, the levelset equation is 'hyperbolic', 
            a subclass of Hamilton-Jacobian equation. 
            
            - 2: if F depends on derivatives of order >= 2 (eg. F = alpha*curvature),
            then the information propagates from all directions, and we can use the 
            central finite difference method to compute the spatial gradients.
        """
        assert pde_class in ['hyperbolic','parabolic'], \
        f"pde_class must be either 1 or 2: {pde_class}"
        
        if pde_class == 'hyperbolic':
            dxb, dxf, dyb, dyf = self.get_bf_gradients()
            
            S = np.sign(F)
            dx = np.maximum(S*dxb, -S*dxf)
            dy = np.maximum(S*dyb, -S*dyf)

            dmag = np.sqrt(dx**2 + dy**2)
            
        else :#pde_class == 'parabolic':
            dx,dy = self.get_central_gradients()
            dmag = comput_mag(dx,dy)# todo
        
        
        # update phi
        dphi = dt* dmag * F
        self.delta = dphi.sum() / dphi.size
        self.grid -= dphi 
        
        # update time
        self.t += dt
    def advect(self, V, dt):
        """
        Args:
        - V (ndarray of shape (w,h,2)): containing x and y component of the vector field
        - dt (float): time step size
        """
        pass
    
    def reinit(self, method='sweep'):
        """
        Reset current grid (phi function) to satisfy Eikonal equality
        in Eqn. 4.12
        
        - method 
            - 'pde': solve eqn. 4.37 with current grid, until steady state
            - 'fmm': fast marching method
            - 'sweep' (default): paper [88]
            - 'exact': paper [64]
            
            Default is 'sweep'
        """
        pass

    def get_bf_gradients(self, switch=True):
        return gradient(self.grid, switch)
    
    def get_curvature(self):
        return curvature(self.grid)
    
    @staticmethod
    def get_test_grid():
        return np.array([[1, 2, 5, 10, 100],
                         [0, -1, 10, -3, 9],
                        [100, -20, 8, 10,-10]], dtype = np.float)

    

In [ ]:
xs,ys,zz = LSTestSample.linear_array()
hv.Image((xs,ys,zz))

In [ ]:
class LSEvolver(CartesianGrid):
    """
    EvolvingLS
    A levelSet evolution according to an initial-valued problem given by a PDE
    Args:
    - F (callable): takes a LevelSet object and time index and returns a np array 
    with the same shape as the levelset's grid
    """
    def __init__(self, xs, ys, data=None, t=0):
        super().__init__(xs, ys, data)
        
        self.time = t #current time
        self.delta = np.inf # average change of LS function values between consecutive time stamps
            
    def run(self, F, dt, pde_class, threshold=1e-3, maxIter=1e4):
        count = 0
        deltas = []
        phis = {}
        while self.delta > threshold:
            if count > maxIter: 
                print("MaxIter reached: ", count)
                break
            self.propagate(F,dt,pde_class)
            deltas.append(self.delta)
            count += 1
            if count%1000:
                phis[self.time] = self.data
        print(f"Ran for {count} steps, for {self.time} periods")
        print(f"\taverage delta phi: {self.delta}")
        return deltas, phis

    def propagate(self, F, dt, pde_class):
        """
        Equation 4.8 and 4.20
        For stability in computing the spatial gradients, use Eqn. 4.33
        
        pde_class (str): 'hyperbolic', 'parabolic'
            - 1: if F depends on at most order 1 derivatives of the levelset function phi 
            wrt space and time, the information propagation has a specific direction 
            (ie. "characteristics"), and we need to be careful about which gradient to 
            take -- backward, forward.  In this case, the levelset equation is 'hyperbolic', 
            which is a subclass of Hamilton-Jacobian equation. 
            
            - 2: if F depends on derivatives of order >= 2 (eg. F = alpha*curvature),
            then the information propagates from all directions, and we can use the 
            central finite difference method to compute the spatial gradients.
        """
        if dt > min(self.dx, self.dy):
            #todo: print error but then make dt smaller smartly
            raise ValueError('dt should be smaller than x and y sample resolutions: ', dt)
                
        assert pde_class in ['hyperbolic','parabolic'], \
        f"pde_class must be either 1 or 2: {pde_class}"
        
        if pde_class == 'hyperbolic':
            dxb, dxf, dyb, dyf = self.get_diff1_bf()
            debug = (
                hv.Image(self.data, label='phi') + hv.Image([])
                + hv.Image(dxb, label='dx back') + hv.Image(dxf, label='dx forward')
                + hv.Image(dyb, label='dy back') + hv.Image(dyf, label='dy forward')
            ).cols(2)
            display(debug)

            S = np.sign(F)
            dx = np.maximum(S*dxb, -S*dxf)
            dy = np.maximum(S*dyb, -S*dyf)

            dmag = np.sqrt(dx**2 + dy**2)
            
        else :#pde_class == 'parabolic':
            dx,dy = self.get_diff1_central()
            dmag = comput_mag(dx,dy)# todo
        
        
        # update phi
        dphi = dt* dmag * F
        self.delta = dphi.sum() / dphi.size
        self.data -= dphi 
        
        # update time
        self.time += dt
        
    def advect(self, V, dt):
        """
        Args:
        - V (ndarray of shape (w,h,2)): containing x and y component of the vector field
        - dt (float): time step size
        """
        pass
    
    def reinit(self, method='sweep'):
        """
        Reset current phi values (in self.data) to satisfy Eikonal equality
        in Eqn. 4.12
        
        - method 
            - 'pde': solve eqn. 4.37 with current phi data, until steady state
            - 'fmm': fast marching method
            - 'sweep' (default): paper [88]
            - 'exact': paper [64]
            
            Default is 'sweep'
        """
        pass

    def get_diff1_bf(self, switch=True):
        dxb, dxf, dyb, dyf = calc.diff1_bf(self.data, switch)
        return dxb/self.dx, dxf/self.dx, dyb/self.dy, dyf/self.dy
    
    def get_diff1_central(self):
        dx, dy= calc.diff1_central(self.data)
        return dx/self.dx, dy/self.dy
    
    def get_curvature(self):
        return curvature(self.data)
    


In [ ]:
xs,ys,zz = LSTestSample.linear_array()
ls = LSEvolver(xs,ys,zz)


In [ ]:
ls

In [ ]:
dx,dy = ls.get_diff1_central()
dxb, dxf,dyb,dyf = ls.get_diff1_bf()

In [ ]:
(
    hv.Image(zz,label='initial ls') 
 + hv.Image(dx, label='initial dx central') 
 + hv.Image(dy, label='initial dy central')
).cols(2);

### Test for debuging diff1_central's dy sign flip


In [ ]:
(
    hv.Image(dxb,label='initial dx back') 
     + hv.Image(dxf, label='initial dx forward') 
     + hv.Image(dyb, label='initial dy back')
     + hv.Image(dyf, label='initial dy forward')

).cols(2)

In [ ]:
tempy, tempx = np.gradient(zz)
(
    hv.Image(zz) 
    + hv.Image(tempx/ls.dx, label='dx') 
    + hv.Image(-tempy/ls.dy, label='dy')
).cols(1)

Yay! Fixed the sign bug. The problem was the np.gradient's positive yaxis is the opposite of Cartesian coordinate's positive yaxis. The later is the direction our underlying dataset (`ls.data`) is referenced to.

Moving on the to the next step..

### Check propagate method

In [ ]:
F = 1
dt = 1e-4
ls.propagate(F, dt, 'hyperbolic')


todo:
- do this in arya
- time profiling, why does it take so long?
     - try with smaller maxiter

In [ ]:
def test_levelset():
    xs,ys,zz = LSTestSample.unit_star1()
    ls = LSEvolver(xs,ys,zz)
    F = 1
    dt = 1e-4
    print("Initial t,delta: ", np.around(ls.time,1), np.around(ls.delta,3))
    
    deltas, phis = ls.run(F, dt, pde_class='hyperbolic', threshold=1e-9)
    print("Final t,delta: ", np.around(ls.time,1), np.around(ls.delta,3))
    plt.plot(deltas)
test_levelset()

Note the importance of `dt` in the delta graph. Try it with different values of dt, eg: `dt = =1, 0.5, 0.01, 0.001, 1e-4`.

Now, let's try it on some signed distance functions.

In [ ]:
def test_levelset_unit_circle():
    n_points = 100
    xlim = (-2,2)
    ylim = (-2,2)
    sdf = sdfs.sdUnitCircle
    xs = np.linspace(*xlim,n_points)
    ys = np.linspace(*ylim,n_points)[::-1]
    zz = sdfs.eval_sdf(xs,ys,sdf)
    base = hv.Image((xs,ys,zz))

    
    ls = LSEvolver(xs,ys,zz)
    F = 1
    dt = 1e-3

    times = [ls.time]
    deltas = [ls.delta]
    phis = {ls.time : ls.data}
    for i in range(100):
        ls.propagate(F, dt=0.3, pde_class='hyperbolic')
        times.append(ls.time)
        deltas.append(ls.delta)
        phis[ls.time] = ls.data.copy()
#     plt.plot(deltas)
    
    dmap = hv.DynamicMap(
    lambda t: hv.operation.contours(hv.Image((xs,ys,phis[t])),levels=10),
    kdims='t')
    dmap = dmap.redim.values(t=list(phis.keys())).opts(
    opts.Contours(contour_opts), 
    opts.Image(img_opts))
    
    
    # show results
    plt.plot(deltas)
    display (
    base.opts(img_opts) 
    * dmap.opts(contour_opts)
    )
    return deltas, base, dmap

AWESOME!!!!!!!!

In [ ]:
deltas, base, dmap = test_levelset_unit_circle()

In [ ]:
plt.plot(deltas)

In [ ]:
(
    base.opts(img_opts) 
    * dmap.opts(contour_opts)
)
dmap#.opts(contour_opts)

In [ ]:
def test_levelset_unit_star1():
#     xs, ys, zz = LSTestSample.unit_star1()
#     xs, ys, zz = LSTestSample.unit_star2()
    xs,ys,zz = LSTestSample.linear_array()


    base = hv.Image((xs,ys,zz))


    ls = LSEvolver(xs,ys,zz)
    F = -1
    dt = 0.3

    times = [ls.time]
    deltas = {ls.time: ls.delta}
    phis = {ls.time : ls.data}
    for i in range(100):
        ls.propagate(F, dt=dt, pde_class='hyperbolic')
        times.append(ls.time)
        deltas[ls.time] = ls.delta
        phis[ls.time] = ls.data.copy()

    #holoviews elements
    deltas.pop(0) # for yrange when visualized
    curve_deltas = hv.Curve(deltas).opts(curve_opts)
    dmap_contours = hv.DynamicMap(lambda t: hv.operation.contours(hv.Image((xs,ys,phis[t])),levels=0),
                                  kdims='t')
    dmap_contours = dmap_contours.redim.values(t=list(phis.keys())).opts(contour_opts,img_opts)
    
    dmap_tpoint= hv.DynamicMap(lambda t: hv.Points( [(t, deltas[t])]), kdims='t')
    dmap_tpoint = dmap_tpoint.redim.values(t=list(deltas.keys())).opts(curve_opts)
                      
    # show results
    overlay = (base*dmap_contours + curve_deltas * dmap_tpoint).opts(shared_axes=False)
    display(overlay)
    
    return deltas, base, dmap_contours, curve_deltas, dmap_tpoint

In [ ]:
deltas, base, dmap_contours, curve_deltas, dmap_tpoint= test_levelset_unit_star1();


In [ ]:
dmap_contours

In [ ]:
(base*dmap_contours + curve_deltas * dmap_tpoint).opts(shared_axes=False)

In [ ]:
n_points = 100
xlim = (-2,2)
ylim = (-2,2)
# sdf = sdfs.sdUnitCircle
# sdf = sdfs.sdUnitHline
sdf = sdfs.sdStar1
# sdf = sdfs.sdStar2



xs = np.linspace(*xlim,n_points)
ys = np.linspace(*ylim,n_points)
zz = sdfs.eval_sdf(xs,ys,sdf)

# plt.imshow(zz.squeeze(), cmap='gray')


In [ ]:
ls = LevelSet(grid=zz)
F = 1
dt = 1
base = hv.Image((xs,ys,ls.grid)).opts(img_opts)

In [ ]:
times = [ls.t]
deltas = [ls.delta]
grids = {ls.t: ls.grid}
for i in range(100):
    ls.propagate(F,dt=0.3, pde_class='hyperbolic')
    times.append(ls.t)
    deltas.append(ls.delta)
    grids[ls.t]=ls.grid.copy()

In [ ]:
plt.plot(deltas)

In [ ]:
dmap = hv.DynamicMap(
    lambda t: hv.operation.contours(hv.Image((xs,ys,grids[t])),levels=10),
    kdims='t')

In [ ]:
base * dmap.redim.values(t=list(grids.keys())).opts(
    opts.Contours(contour_opts),
    opts.Image(img_opts)
).opts(legend_position='bottom')


In [ ]:
dmap.redim.range(t=(0.001).opts(contour_opts)

In [ ]:
dmap = hv.DynamicMap(
    lambda tidx: hv.operation.contours(hv.Image((xs,ys,temp[tidx])),levels=0),
    kdims='tidx')

---
Modified: Jul 22, 2019
## todo: open a git issue on holoviews

### For git issue

In [ ]:
hv.NdOverlay(contours).options({'Contours': {'color': 'Element'}})
# hv.NdOverlay(contours).opts(hv.opts('Contours', color='Element'))

In [ ]:
hv.NdOverlay(contours).opts(hv.opts('Contours', color='Element'))

In [ ]:
print(tmp)

---
### Colormaps

In [ ]:
cmaps = list(hv.Palette.colormaps.keys())[1:]
cmaps_filtered = [cmap for i,cmap in enumerate(cmaps) if i%3]

print(cmaps_filtered)

Other useful commands
- resource: https://is.gd/1WFERD

In [ ]:
# cmaps = list(hv.Palette.colormaps.keys())[1:]
# cmaps_filtered = [cmap for i,cmap in enumerate(cmaps) if i%3]
# from holoviews.plotting import list_cmaps
# cmaps_filtered = list_cmaps(category='Mono Sequential', reverse=False)

In [ ]:
# nd_contours.opts(opts.Contours(color=hv.Cycle(['red','green','blue'])))
nd_contours.options({'Contours': dict(cmap=hv.Cycle(cmaps_filtered))})

In [ ]:
# cmaps_filtered is a list of cmaps I chose manually
for step_idx in range(10):
    img = hv.Image( (xs, ys, zz), group=f'{step_idx}' )
    contour = hv.operation.contours(zz_img, levels=0).relabel(group=f'{step_idx}') \
                .opts(xlim=xrange, ylim=yrange, cmap=cmaps_filtered[step_idx%len(cmaps_filtered)])

    contours[step_idx] = contour
nd_contours = hv.NdOverlay(contours, group='Contours', kdims='step')



In [ ]:
contours_cmapped = {}
for cmap in cmaps:
    try: 
        contours_cmapped[cmap]=hv.operation.contours(hv_img).opts(cmap=cmap)
    except:
        print('**failed: ', cmap)
cmap_filtered = [cmap for i,cmap in enumerate(cmaps) if i%3]
    


In [ ]:
hv.NdLayout({k:v for i,(k,v) in enumerate(contours_cmapped.items()) if i%3})

---
## Snippets 
- for gradient computation and visualization


In [ ]:

# compute gradients
gradx = correlate2d(zz, kernel, mode='same')
grady = correlate2d(zz, kernel.T, mode='same')

ang, mag = u.UV2angMag(gradx, grady)

gradfield = hv.VectorField((xs, ys, ang, mag)).opts(vfield_opts)
gradmag_img = hv.Image( (xs,ys,np.abs(mag)), group='grad', label='t0' ).opts(img_opts).opts(xlim=xrange, ylim=yrange)

overlay = datashade(zz0_img, cmap=GnBu9) * zz0_contour * gradfield + gradmag_img

In [ ]:
import imageio as iio

In [ ]:
test_img = iio.imread('../data/test/gradient.jpg')
bounds
print(test_img.shape)

In [ ]:
test_x, test_y = np.meshgrid(range(test_img.shape[1]), range(test_img.shape[0]))
test_x.shape, test_y.shape
# hv.RGB(test_img. 

In [ ]:
r,g,b = np.dsplit(test_img, test_img.shape[-1])
bounds = (0, 0, test_x.shape[1], test_x.shape[0])
(
    hv.Image( r.squeeze(), bounds=bounds)
    +  hv.Image( g.squeeze(), bounds=bounds)
    +  hv.Image( b.squeeze(), bounds=bounds)
).cols(1).opts(
    opts.Image(height=test_x.shape[0], width=test_x.shape[1])
)

In [ ]:
import vec2 as vec2 

In [ ]:
del vec2

In [ ]:
type(vec2)

In [ ]:
class CurveSimulator(param.Parameterized):

    n_steps = param.Integer(label='Number of simulation steps', default=100)
    p = param.ObjectSelector(label='p', default=0., objects=np.linspace(0,1,num=n_steps.default))
    reset = param.Action(lambda x: x.reset_handler(), doc="Click to clear the buffer and reset p")
    t_interval = param.Number(label='t_interval', doc='Time interval between plotting two points',
                              softbounds=(0., 5.),
                              default=0.)

    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,width=W, height=H, 
                             xlim=(-1,1), ylim=(-1,1),
                             color=dim('p')*256-50,
                             tools=['hover']
                            )
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    
    
    ################################################################################
    # Parameter Dependencies
    ################################################################################    
    @param.depends('n_steps', watch=True)
    def _update_p(self):
        self.count['p'] += 1
        self.param['p'].objects = np.linspace(0,1,num=self.n_steps)
        print('updated p with new number of simulation steps: ', self.n_steps)
    
    @param.depends('p', watch=True)
    def send_point(self):
        point = pd.DataFrame([(self.p, *self.cfunc(self.p))], columns=['p','x','y'])
        self.data_src.emit(point)
        time.sleep(self.t_interval)
        
    def reset_handler(self):
        self.set_param(p=0.0)
        self.dfstream.clear()

        
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, cfunc, n_steps=100, **kwargs):
        """
        Args:
        - cfunc (function): given an input of a float p in [0,1], returns (x,y), a 
        tuple of x and y coords
        
        - n_steps (int): number of simulation steps along the range of [0,1] for 
        the parameter, p
        """
        super().__init__(**kwargs) # this is super important
        self.cfunc = cfunc 
        self.n_steps = n_steps
        
        self.example = pd.DataFrame({'p': [], 'x':[], 'y':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=min(self.n_steps, 100), index=False)
        self.set_dmap_curve()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
    

    def set_dmap_curve(self):
        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream])#.opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts)
        
    def set_dmap_x(self):
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
    
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    def viewable(self):
        return self.overlay
        


### Define curve function

In [ ]:
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
# Try different functions
xfunc = lambda p: np.sin(2*np.pi*p)**2
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
xfunc = lambda p: np.sin(2*np.pi*p)**10
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
# Alternating along a straight line 
xfunc = lambda p: np.sin(2*np.pi*p)**2
yfunc = lambda p: np.cos(2*np.pi*p)**2

In [ ]:
# Doesn't have to choose a periodic function
xfunc = lambda p: np.log(p)
yfunc = lambda p: p

In [ ]:
# Something happens at p=0.5
xfunc = lambda p: np.sin(2*np.pi*p**2)*p**3
yfunc = lambda p: np.sin(np.pi*p**0.5)

### Create the simulator for the curve

In [ ]:
cfunc = lambda p: (xfunc(p), yfunc(p))
c = CurveSimulator(cfunc)

### Show the simulator

In [ ]:
pn.Row(
    pn.Param(c.param, width=500, widgets={
        'p': pn.widgets.DiscretePlayer,
        'reset': pn.widgets.Button(name=c.param['reset'].label),
        't_interval': pn.widgets.FloatSlider
    }),
    pn.panel(c.viewable())
)

In [ ]:
c.dfstream.data